In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Model
import keras_tuner

data_path = "./dataset"

physical_devices = tf.config.list_physical_devices("GPU")
print(f"GPUs: {len(physical_devices)}")

Using TensorFlow backend
GPUs: 1


### Load Model

In [2]:
model = keras.models.load_model('./bot/fish_model')

In [3]:
img_width = 200
img_height = 100
batch_size = 32

test_data = keras.preprocessing.image_dataset_from_directory(
    data_path + "/real_test_set",
    label_mode="categorical",
    image_size=(img_width, img_height),
)

test_images, test_labels = [], []



for batch in test_data:
    X, y = batch
    test_images.append(X)
    test_labels.append(y)



test_images = np.vstack(test_images)
test_labels = np.vstack(test_labels)


y_predict = model.predict(test_images)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_predict]

Found 3404 files belonging to 2 classes.
107/107 [==============================] - 4s 10ms/step


In [ ]:
from plotly.subplots import make_subplots
import numpy as np
from sklearn.metrics import accuracy_score


index = np.arange(len(test_labels))

fig = make_subplots(rows=1, cols=1, subplot_titles=("Fish"))

fig.add_scatter(
    x=index,
    y=test_labels[:, 0],
    mode="markers",
    name="fisch",
    marker_color="rgba(255, 0, 0, .6)",
    row=1,
    col=1,
)
fig.add_scatter(
    x=index,
    y=y_predict[:, 0],
    mode="markers",
    name="preditcion",
    marker_color="rgba(0, 0, 255, .6)",
    row=1,
    col=1,
)

fig.update_layout(
    title=f"Accuracy {accuracy_score(test_labels[:,:1],y_pred_binary)*100:.2f} %",
    yaxis_zeroline=False,
    xaxis_zeroline=False,
)
fig.show()

In [5]:
wrong_predictions = np.where(np.int8(test_labels[:, :1])[:, 0] != y_pred_binary)[0]

In [ ]:
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt

idx = 213
threshold = 0.01


def get_activations(model, layer_name, input_data):
    activation_model = Model(
        inputs=model.input, outputs=model.get_layer(layer_name).output
    )
    activations = activation_model.predict(input_data)
    return activations

def generate_heatmap(activations):
    heatmap = np.mean(
        activations, axis=-1
    )
    return heatmap

for idx in wrong_predictions:
    input_data = test_images[idx]
    activations = get_activations(model, "pool2", np.expand_dims(input_data, axis=0))

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(1 / 255 * test_images[idx, :, :, :])
    axes[0].set_title(f"y: {test_labels[idx, 0]} pred: {y_predict[idx, 0]:.2f}")

    heatmap = generate_heatmap(activations)
    axes[1].imshow(heatmap[0], cmap="viridis")
    axes[1].set_title(f"activation region")

    y1, x1 = np.where(heatmap[0] > np.mean(heatmap[0]))

    scale_x = 4.55
    scale_y = 4.255
    rect_width = int((max(x1) - min(x1)) * scale_x) + 2
    rect_height = int((max(y1) - min(y1)) * scale_y) + 2

    rect = Rectangle(
        (int(min(x1) * scale_x) - 1, int(min(y1) * scale_y) - 1),
        rect_width,
        rect_height,
        fill=False,
        edgecolor="red",
        linewidth=2,
    )

    axes[2].imshow(1 / 255 * test_images[idx, :, :, :])
    axes[2].add_patch(rect)
    axes[2].set_title(f"activation region")

    plt.show()